# Cartel Detection using Network Analysis

This project attempts to demonstrate the use of graph theory tools in identifying possible cartel conduct among hypothetical players in a dummy procurement dataset. Data was randomly generated. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import matplotlib.colors as mcolors
from pyvis.network import Network
from read_data import load_data
df = load_data()

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              5000 non-null   datetime64[ns]
 1   PROJECT_ID        5000 non-null   int64         
 2   PROJECT_TYPE      5000 non-null   object        
 3    ABC              5000 non-null   object        
 4   NUM_BIDS          5000 non-null   int64         
 5   WINNER_NAME       5000 non-null   object        
 6    CONTRACT_PRICE   5000 non-null   object        
 7   DURATION          5000 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 312.6+ KB


In [3]:
#create subsets for the dataset by project type
df_infra = df[df['PROJECT_TYPE'] == 'Infrastructure Projects']
df_goods = df[df['PROJECT_TYPE'] == 'Goods']
df_const = df[df['PROJECT_TYPE'] == 'Consulting Services']


Let's focus on Infrastructure Projects for now:

In [4]:
df_infra

,DATE,PROJECT_ID,PROJECT_TYPE,ABC,NUM_BIDS,WINNER_NAME,CONTRACT_PRICE,DURATION
0,2020-01-01,3602,Infrastructure Projects,"1,606,239,455.00",3,Denji,"1,606,232,762.00",33
2,2020-01-01,3642,Infrastructure Projects,"198,074,848.00",2,Makima,"198,078,287.00",13
5,2020-01-02,966,Infrastructure Projects,"1,938,249,259.00",1,Denji,"1,938,241,775.00",27
9,2020-01-03,3386,Infrastructure Projects,"1,683,883,574.00",1,Denji,"1,683,874,917.00",24
10,2020-01-04,683,Infrastructure Projects,"1,883,993,509.00",8,Pochita,"1,884,001,597.00",14
...,...,...,...,...,...,...,...,...
4987,2023-07-30,392,Infrastructure Projects,"1,177,296,211.00",3,Himeno,"1,177,288,674.00",18
4990,2023-07-30,1758,Infrastructure Projects,"1,219,243,055.00",9,Denji,"1,219,243,807.00",23
4991,2023-07-30,2247,Infrastructure Projects,"779,326,959.00",9,Mikata,"779,331,374.00",23
4995,2023-07-31,977,Infrastructure Projects,"1,512,354,597.00",1,Reze,"1,512,346,313.00",15
